# 2024C1-02 SMI WAXS TReXS processing notebook 

## Imports

In [1]:
import pathlib
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from PIL import Image
from tqdm.auto import tqdm 

import fabio
from smi_analysis import SMI_beamline

ERROR:pygix.transform:Unable to import pyFAI.ocl_azim_csr: cannot import name 'ocl_azim_csr' from 'pyFAI' (/nsls2/conda/envs/2024-1.0-py311-tiled-rc1/lib/python3.11/site-packages/pyFAI/__init__.py)
ERROR:pygix.transform:Unable to import pyFAI.ocl_azim_lut for: cannot import name 'ocl_azim_lut' from 'pyFAI' (/nsls2/conda/envs/2024-1.0-py311-tiled-rc1/lib/python3.11/site-packages/pyFAI/__init__.py)
ERROR:pygix.transform:Unable to import pyFAI.ocl_sort for: cannot import name 'ocl_sort' from 'pyFAI' (/nsls2/conda/envs/2024-1.0-py311-tiled-rc1/lib/python3.11/site-packages/pyFAI/__init__.py)


## Define paths & functions

In [2]:
# sample name dict
sn = {
    'SiN-0': 'Bare_SiN',
    'SiN-1': 'PM6_10CN-CB_SiN',
    'SiN-2': 'PM6_10CN-CF_SiN',
    'SiN-3': 'PM6_CB_SiN',
    'SiN-4': 'PM6_CF_SiN',
    'SiN-7': 'PM6_p5CN-2CF-3CB_SiN',
    'SiN-8': 'PM6-Y6BO_CB_SiN',
    'SiN-9': 'PM6-Y6BO_p5CN-CB_SiN',
    'SiN-10': 'PM6-Y6_CB_SiN',
    'SiN-11': 'PM6-Y6_p5CN-CB_SiN'
}

In [3]:
# Define paths

propPath = pathlib.Path('/nsls2/data/smi/proposals/2024-1/pass-313064')
rawPath = propPath.joinpath('900KW')

### SMI function

In [4]:
def waxs_SMI_numpy_loading(path, filename_sublists):
    """
    Function adapted from Guillames SMI notebooks to process raw waxs tiffs:
    
    Returns filename_wa0_list, recip_list, recip_extents, caked_list, caked_extents
    """
    
    #waxs
    geometry = 'Transmission'
    bs_kind = 'pindiode'
    incident_angle = np.deg2rad(0)

    #WAXS
    detector_waxs = 'Pilatus900kw'
    sdd_waxs = 280
    center_waxs = [97, 1255.9]
    bs_pos_waxs = [[97, 1070], [0, 0], [0, 0]]
    
    flatPath = pathlib.Path('/nsls2/data/smi/legacy/results/analysis/2024_1/314483_Freychet_04')
    flatfield = np.rot90(fabio.open(flatPath.joinpath('GF_Flatfield_Sedge_uhighg1600eV_10s_wa20_2477eV_pffBT4T_id481136_000000_WAXS.tif')).data, 1)

    
    filename_wa0_list = []
    recip_list = []
    recip_extents = []
    caked_list = []
    caked_extents = []
    for dat in tqdm(filename_sublists, desc='Processing tiffs'):
        
            waxs_angle = [np.deg2rad(-0.06), np.deg2rad(19.7-0.06)]
            # print(dat)

            idx = dat[0].find('eV')
            energy = 0.001*float(dat[0][idx-7:idx])
            # print(energy)
            wav = 1E-10 * (12.398/energy)

            #This part is to stitch the data
            SMI_waxs = SMI_beamline.SMI_geometry(geometry = geometry,
                                                 detector = detector_waxs,
                                                 sdd = sdd_waxs,
                                                 wav = wav,
                                                 alphai = 0,
                                                 center = center_waxs,
                                                 bs_pos = bs_pos_waxs,
                                                 det_angles = waxs_angle,
                                                 bs_kind = None)


            SMI_waxs.open_data(path, dat)
            
            SMI_waxs.masks[1][1254, 47], SMI_waxs.masks[1][1255, 47]=True, True
            SMI_waxs.masks[4][1254, 47], SMI_waxs.masks[4][1255, 47]=True, True

            SMI_waxs.masks[1][979:1050, 0:100], SMI_waxs.masks[4][979:1050, 0:100]=True, True
            SMI_waxs.masks[1][967, 67], SMI_waxs.masks[4][967, 67]=True, True

            SMI_waxs.masks[1][490:555, 100:], SMI_waxs.masks[4][490:555, 100:]=True, True

            SMI_waxs.masks[1][1231, 174], SMI_waxs.masks[1][1232, 174]=True, True
            SMI_waxs.masks[4][1231, 174], SMI_waxs.masks[4][1232, 174]=True, True

            SMI_waxs.masks[1][1414, 179], SMI_waxs.masks[1][1415, 179]=True, True
            SMI_waxs.masks[4][1414, 179], SMI_waxs.masks[4][1415, 179]=True, True

            SMI_waxs.masks[1][858, 5], SMI_waxs.masks[1][859, 5]=True, True
            SMI_waxs.masks[4][858, 5], SMI_waxs.masks[4][859, 5]=True, True

            SMI_waxs.masks[1][414, 6], SMI_waxs.masks[4][414, 6], =True, True
            SMI_waxs.masks[1][394, 138], SMI_waxs.masks[4][394, 138]=True, True

            SMI_waxs.masks[1][364, 41], SMI_waxs.masks[1][365, 41]=True, True
            SMI_waxs.masks[4][364, 41], SMI_waxs.masks[4][365, 41]=True, True

            SMI_waxs.masks[1][364, 96], SMI_waxs.masks[1][365, 96]=True, True
            SMI_waxs.masks[4][364, 96], SMI_waxs.masks[4][365, 96]=True, True

            SMI_waxs.masks[1][304, 96], SMI_waxs.masks[1][305, 96], SMI_waxs.masks[1][304, 97], SMI_waxs.masks[1][305, 97]=True, True, True, True
            SMI_waxs.masks[4][304, 96], SMI_waxs.masks[4][305, 96], SMI_waxs.masks[4][304, 97], SMI_waxs.masks[4][305, 97]=True, True, True, True

            SMI_waxs.masks[1][181, 97], SMI_waxs.masks[1][182, 97], SMI_waxs.masks[1][181, 98], SMI_waxs.masks[1][182, 98]=True, True, True, True
            SMI_waxs.masks[4][181, 97], SMI_waxs.masks[4][182, 97], SMI_waxs.masks[4][181, 98], SMI_waxs.masks[4][182, 98]=True, True, True, True

            for i, (img, mask) in enumerate(zip(SMI_waxs.imgs, SMI_waxs.masks)):
                if i%3==0:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, :195]<1)] = True

                if i%3==1:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, 211:406]<1)] = True

                if i%3==2:
                    SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7
                    SMI_waxs.masks[i][np.where(flatfield[:, -195:]<1)] = True

            SMI_waxs.stitching_data(interp_factor=3, flag_scale=False)
            
            SMI_waxs.caking()
            
            filename_wa0_list.append(dat[0])
            recip_list.append(SMI_waxs.img_st)
            recip_extents.append([SMI_waxs.qp[0], SMI_waxs.qp[-1], SMI_waxs.qz[0], SMI_waxs.qz[-1]])
            
            caked_list.append(SMI_waxs.cake)
            caked_extents.append([SMI_waxs.q_cake[0], SMI_waxs.q_cake[-1], SMI_waxs.chi_cake[0], SMI_waxs.chi_cake[-1]])
            
    return filename_wa0_list, recip_list, recip_extents, caked_list, caked_extents

## Load data & save zarrs

In [23]:
filenames = [f.name for f in rawPath.glob('TC_SiN-3*')]
f = filenames[0]
# f.split('_')
float(f[f.find('_id')+1:].split('_')[0][2:])

599762.0

In [29]:
def get_id(f):
    return float(f[f.find('_id')+1:].split('_')[0][2:])

# sorted(filenames, key=lambda x: float(x[x.find('_id')+1:].split('_')[0][2:]))
sorted(filenames, key=lambda x: get_id(x))

['TC_SiN-3_prs0deg_sdd1.8m_2445.00eV_wa0_bpm1.462_id598958_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2450.00eV_wa0_bpm1.497_id598959_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2455.00eV_wa0_bpm1.522_id598960_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2460.00eV_wa0_bpm1.541_id598961_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2465.00eV_wa0_bpm1.570_id598962_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2470.00eV_wa0_bpm1.601_id598963_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2472.00eV_wa0_bpm1.600_id598964_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2473.00eV_wa0_bpm1.593_id598965_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2474.00eV_wa0_bpm1.620_id598966_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2474.50eV_wa0_bpm1.601_id598967_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2475.00eV_wa0_bpm1.609_id598968_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2475.50eV_wa0_bpm1.615_id598969_000000_WAXS.tif',
 'TC_SiN-3_prs0deg_sdd1.8m_2476.00eV_wa0_bpm1.611_id598970_000000_WAXS.tif',

In [ ]:
# all_saxs = set(waxsPath1.glob('*.tif'))
# test_saxs = set(waxsPath1.glob('test*'))
# sample_names = sorted(set([f.name[3:f.name.find('_sdd1.8')] for f in all_saxs.difference(test_saxs)]))[:-1]
# sample_names

In [6]:
unique_sample_numbers = sorted(set([f.name.split('_')[1] for f in rawPath.glob('TC*')]), key=lambda x: float(x.split('-')[-1]))
unique_sample_rotations = sorted(set([f.name.split('_')[2] for f in rawPath.glob('TC*')]))

In [ ]:
# filename_list = [f.name for f in sorted(waxsPath.glob('*Trmsn_18*prs35*'))]

# # Make sublists (of sample filenames)
# group_size = 2
# filename_sublists = [filename_list[i:i + group_size] for i in range(0, len(filename_list), group_size)]

In [ ]:
# filename_sublists

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
def remove_duplicate_energies(filename_list):
    file_energies = []
    kept_files = []

    for filename in filename_list:
        file_energy = filename[:filename.find('eV')].split('_')[-1]
        if file_energy in file_energies:
            pass
        else:
            file_energies.append(file_energy)
            kept_files.append(filename)

    return kept_files

In [ ]:
for sample_number in unique_sample_numbers[1:2]:
    all_paths = set(rawPath.glob(f'TC*_{sample_number}_*prs0deg*'))
    display([f.name for f in sorted(all_paths, key=lambda x: x.stem.split('_')[7])])
    print(len([f.name for f in sorted(all_paths, key=lambda x: x.stem.split('_')[7])]))

In [ ]:
# 598792
for sample_number in unique_sample_numbers[1:2]:
    all_paths = set(rawPath.glob(f'TC*_{sample_number}_*prs0deg*'))
    all_paths = [f.name for f in sorted(all_paths, key=lambda x: x.stem.split('_')[7]) 
                 if float(f.stem.split('_')[7][2:]) > 598792]
    print(len(all_paths))
    display(all_paths)

In [ ]:
# We need to check that the number of energies for the 2 detector positions is consistent (63)
wa0_filename_list = [fname for fname in filenames if 'wa0' in fname]
wa20_filename_list = [fname for fname in filenames if 'wa20' in fname]
if len(wa0_filename_list) == num_energies:
    pass
else:
    wa0_filename_list = remove_duplicate_energies(wa0_filename_list)

if len(wa20_filename_list) == num_energies:
    pass
else:
    wa20_filename_list = remove_duplicate_energies(wa20_filename_list)

In [ ]:
len(wa0_filename_list)

In [ ]:
len(wa20_filename_list)

In [ ]:
filename_list = sorted(wa0_filename_list + wa20_filename_list)
# len(filename_list)
# Make sublists to stitch two waxs positions together
group_size = 2
filename_sublists = [filename_list[i:i + group_size] for i in range(0, len(filename_list), group_size)]

In [ ]:
len(filename_sublists)

In [ ]:
all_paths

In [ ]:
unique_sample_rotations

In [ ]:
all_paths

In [7]:
# Cell for scans with incident angle recorded in name

num_energies = 39

for sample_number in tqdm(unique_sample_numbers[1:], desc='Samples'):
    for sample_rotation in tqdm(unique_sample_rotations[1:2], desc='Angles'):

        # Select files for a given sample and rotation
        all_paths = set(rawPath.glob(f'TC*_{sample_number}_*{sample_rotation}_*'))
        
        # For now only select first scans, not the damage test repeats
        # filename_list = [f.name for f in sorted(all_paths)]
        filename_list = [f.name for f in sorted(all_paths, key=lambda x: x.stem.split('_')[7]) 
                         if float(f.stem.split('_')[7][2:]) > 598792]
        
        # We need to check that the number of energies for the 2 detector positions is consistent (63)
        wa0_filename_list = [fname for fname in filename_list if 'wa0' in fname]
        wa20_filename_list = [fname for fname in filename_list if 'wa20' in fname]
        if len(wa0_filename_list) == num_energies:
            pass
        else:
            wa0_filename_list = remove_duplicate_energies(wa0_filename_list)

        if len(wa20_filename_list) == num_energies:
            pass
        else:
            wa20_filename_list = remove_duplicate_energies(wa20_filename_list)
            
        filename_list =  sorted(wa0_filename_list + wa20_filename_list)
        
        # Make sublists to stitch two waxs positions together
        group_size = 2
        filename_sublists = [filename_list[i:i + group_size] for i in range(0, len(filename_list), group_size)]

        # Run SMI loading code (this produces some fabio and divide by zero errors)
        names_list, recip_list, recip_extents, caked_list, caked_extents = waxs_SMI_numpy_loading(rawPath, filename_sublists)

        

        # Define naming scheme:
        waxs_naming_scheme = ['project', 'sample_number', 'rotation_from_normal', 'set_sdd', 'energy', 'waxs_det_position',
                              'bpm', 'id', 'misc', 'detector']
        md_naming_scheme = waxs_naming_scheme.copy()


        # Construct xarrays with full values along detector dimensions and the energy dimension
        # They contain sample name and theta value as well, as single values to be concatenated in later steps
        recip_DA_rows = []
        caked_DA_rows = []
        zipped_lists = zip(names_list, recip_list, recip_extents, caked_list, caked_extents)
        for filename, recip_arr, recip_extent, caked_arr, caked_extent in zipped_lists:
            # print(filename)
            # print(recip_arr.shape)
            # print(recip_extent)
            # print(caked_arr.shape)
            # print(caked_extent)

            attr_dict = {}
            md_list = filename.split('_')
            for i, md_item in enumerate(md_naming_scheme):
                attr_dict[md_item] = md_list[i]

            recip_DA = xr.DataArray(data = recip_arr, 
                                    dims = ['pix_y', 'pix_x'],
                                    attrs = attr_dict)
            recip_DA = recip_DA.assign_coords({
                'pix_x': recip_DA.pix_x.data,
                'pix_y': recip_DA.pix_y.data,
                'q_x': ('pix_x', np.linspace(recip_extent[0], recip_extent[1], len(recip_DA.pix_x.data))),
                'q_y': ('pix_y', np.linspace(recip_extent[3], recip_extent[2], len(recip_DA.pix_y.data)))
            })
            recip_DA = recip_DA.expand_dims({
                'energy': [float(recip_DA.energy[:-2])],
                'sample_name': [sn[recip_DA.sample_number]],
                'theta': [90 - float(recip_DA.rotation_from_normal[3:-3])]
            })
            recip_DA_rows.append(recip_DA)

            caked_DA = xr.DataArray(data = caked_arr, 
                                    dims = ['index_y', 'index_x'],
                                    attrs = attr_dict)
            caked_DA = caked_DA.assign_coords({
                'index_x': caked_DA.index_x.data,
                'index_y': caked_DA.index_y.data,
                'q_r': ('index_x', np.linspace(caked_extent[0], caked_extent[1], len(caked_DA.index_x.data))),
                'chi': ('index_y', np.linspace(caked_extent[3], caked_extent[2], len(caked_DA.index_y.data)))
            }) 
            caked_DA = caked_DA.expand_dims({
                'energy': [float(caked_DA.energy[:-2])],
                'sample_name': [sn[caked_DA.sample_number]],
                'theta': [90 - float(caked_DA.rotation_from_normal[3:-3])]
            })
            caked_DA_rows.append(caked_DA)

        recip_DA = xr.concat(recip_DA_rows, 'energy')
        caked_DA = xr.concat(caked_DA_rows, 'energy')

        # Save sample zarr, load later to concatenate full zarr
        sampleZarrsPath = propPath.joinpath('processed_data/andrew/zarrs')

        recip_samp_zarr_name = 'recip_'+recip_DA.sample_name.values[0]+'_'+str(int(recip_DA.theta.values[0]))+'deg.zarr'
        recip_DS = recip_DA.to_dataset(name='flatfield_corr')
        recip_DS.to_zarr(sampleZarrsPath.joinpath(recip_samp_zarr_name), mode='w')

        caked_samp_zarr_name = 'caked_'+caked_DA.sample_name.values[0]+'_'+str(int(caked_DA.theta.values[0]))+'deg.zarr'
        caked_DS = caked_DA.to_dataset(name='flatfield_corr')
        caked_DS.to_zarr(sampleZarrsPath.joinpath(caked_samp_zarr_name), mode='w')

Samples:   0%|          | 0/9 [00:00<?, ?it/s]

Angles:   0%|          | 0/1 [00:00<?, ?it/s]

Processing tiffs:   0%|          | 0/39 [00:00<?, ?it/s]

/tmp/ipykernel_31811/1338544897.py:86: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: divide by zero encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:94: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7
/tmp/ipykernel_31811/1338544897.py:86: RuntimeWarning: divide by zero encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7


Angles:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processing tiffs:   0%|          | 0/39 [00:00<?, ?it/s]

/tmp/ipykernel_31811/1338544897.py:86: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: divide by zero encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:94: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7


Angles:   0%|          | 0/1 [00:00<?, ?it/s]

Processing tiffs:   0%|          | 0/39 [00:00<?, ?it/s]

/tmp/ipykernel_31811/1338544897.py:86: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, :195]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: divide by zero encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:90: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, 211:406]+1E-7
/tmp/ipykernel_31811/1338544897.py:94: RuntimeWarning: invalid value encountered in divide
  SMI_waxs.imgs[i] = SMI_waxs.imgs[i]/flatfield[:, -195:]+1E-7


In [ ]:
caked_DA = xr.open_zarr(sampleZarrsPath.joinpath('caked_PM6_5CN-CB_55deg.zarr'))['flatfield_corr']
caked_DA

In [ ]:
recip_DA.sel(energy=1e10,method='nearest').plot.imshow(vmin=0,vmax=np.percentile(SMI_waxs.img_st, 99), x='q_x', y='q_y')
plt.show()

In [ ]:
caked_DA.squeeze().sel(energy=0,method='nearest').plot.imshow(vmin=0,vmax=np.percentile(SMI_waxs.cake, 97), x='q_r', y='chi')
plt.show()

In [ ]:
# trmsn_35_tot = sorted(reducedPath.glob('*tot*Trmsn_35*.txt'))
# trmsn_35_ver = sorted(reducedPath.glob('*ver*Trmsn_35*.txt'))
# trmsn_35_hor = sorted(reducedPath.glob('*hor*Trmsn_35*.txt'))

# len([f.name for f in trmsn_35_tot])

In [ ]:
# for file in trmsn_35_tot:
#     pr = np.loadtxt(file)
#     plt.plot(pr[:, 1]-0.9*np.mean(pr[1100:1250, 1]))
#     plt.show()
#     plt.close()